In [ ]:
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/merged_labels_from_our_members.csv')

In [ ]:
df['label'].value_counts()

,count
label,
positive,4738
negative,4137
neutral,1637
spam,96


In [ ]:
df.head()

,comment,label
0,"sản phẩm tạm ổn với giá tiền, giao thiếu hàng,...",neutral
1,thảm dày dặn đẹp . trừ 1 sao là giao nhầm màu ...,neutral
2,giao hàng khá nhanh nhé shop.mà hàng mình nhận...,neutral
3,chất lượng sản phẩm:xanh va hong xốp cũng dày ...,positive
4,đúng với mô tả:đúng mô tả chất lượng sản phẩm:...,neutral


In [ ]:
df = df[['comment', 'label']][df['label']!='spam']
df['label'].value_counts()


,count
label,
positive,4738
negative,4137
neutral,1637


In [ ]:
df = df.dropna()
df = df[df['label']!= 'neutral']
df['label'].value_counts()

,count
label,
positive,4738
negative,4137


In [ ]:
# Calculate the number of positive and negative records
positive_count = df[df['label'] == 'positive'].shape[0]
negative_count = df[df['label'] == 'negative'].shape[0]

# Determine the smaller class size
min_class_size = min(positive_count, negative_count)

# Randomly sample from the larger class to match the size of the smaller class
if positive_count > negative_count:
  df_positive_sampled = df[df['label'] == 'positive'].sample(n=min_class_size, random_state=42)
  df_negative = df[df['label'] == 'negative']
  df = pd.concat([df_positive_sampled, df_negative], axis=0)
elif negative_count > positive_count:
  df_negative_sampled = df[df['label'] == 'negative'].sample(n=min_class_size, random_state=42)
  df_positive = df[df['label'] == 'positive']
  df = pd.concat([df_positive, df_negative_sampled], axis=0)
else:
  # If the classes are already balanced, do nothing
  pass

df['label'].value_counts()


,count
label,
positive,4137
negative,4137


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from underthesea import word_tokenize, text_normalize, sentiment

# 2. Split the data into training and testing sets

def preprocess_text(text):
  """Preprocesses Vietnamese text using underthesea.

  Args:
    text: The input Vietnamese text.

  Returns:
    The preprocessed text.
  """

  # 1. Text Normalization:
  text = text.lower()
  text = text_normalize(text)

  # 2. Word Segmentation (Tokenization):
  tokens = word_tokenize(text)

  # 3. (Optional) Remove Stopwords, punctuation, special characters:
  # You may need to define a list of stopwords based on your needs and remove them here
  stop_words = [
    "tôi", "bạn", "chúng tôi", "họ", "nó", "ông", "bà", "cô", "chúng ta", "hắn", "mình",
    "ở", "trong", "ngoài", "trên", "dưới", "với", "đến", "từ",
    "và", "nhưng", "hoặc", "nếu", "vì", "nên", "rồi", "mà", "khi", "sau khi",
    "rất", "cũng", "chỉ", "đã", "đang", "sẽ", "nữa", "mới", "lại", "thế",
    "à", "ơi", "nhé", "hả", "không", "có", "phải", "vậy", "thôi", "được"
]
  tokens = [word for word in tokens if word not in stop_words]

  # 4. Join tokens back into a string:
  preprocessed_text = " ".join(tokens)

  return preprocessed_text

In [ ]:
df['comment'] = df['comment'].apply(preprocess_text)

In [ ]:
df.head()

,comment,label
5410,nền của nhà maybeline thực sự ko j để bàn cãi ...,positive
861,công dụng : ngăn mùi mùi hương : mát lần thứ 2...,positive
3256,đúng mô tả : sp đúng mô tả chất lượng sản phẩm...,positive
4168,thiết kế bìa : đẹp đối tượng độc giả : 18 hàg ...,positive
8861,bàn phím xinh lắm 🥰 . cần khắc phục khâu đóng gói,positive


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['comment'], df['label'], test_size=0.2, random_state=42, stratify=df['label'])
# 3. Convert text data to TF-IDF features
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have your data in X_train_tfidf, X_test_tfidf, y_train, y_test

# Define a list of models to compare
models = [
    ('SVM', SVC(probability = True, kernel='linear', C=1)),
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
    ('Logistic Regression', LogisticRegression(max_iter=1000, random_state=42)),
    ('Multinomial Naive Bayes', MultinomialNB())
]

# Train and evaluate each model
best_model = None
best_accuracy = 0

# Train and evaluate each model
results = []
for name, model in models:
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, zero_division=0)

    results.append((name, accuracy, report))

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model


# Save the best model to a file
if best_model:
    with open('best_model_v3.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    print("Best model saved as best_model_v2.pkl")

with open('vectorizer_v3.pkl', 'wb') as f:
    pickle.dump(tfidf, f)
print("TF-IDF vectorizer saved as vectorizer_v2.pkl")

# Print the results
for name, accuracy, report in results:
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Classification Report:\n{report}\n")

Best model saved as best_model_v2.pkl
TF-IDF vectorizer saved as vectorizer_v2.pkl
Model: SVM
Accuracy: 0.8677
Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.87      0.87       828
    positive       0.87      0.86      0.87       827

    accuracy                           0.87      1655
   macro avg       0.87      0.87      0.87      1655
weighted avg       0.87      0.87      0.87      1655


Model: Random Forest
Accuracy: 0.8260
Classification Report:
              precision    recall  f1-score   support

    negative       0.85      0.80      0.82       828
    positive       0.81      0.85      0.83       827

    accuracy                           0.83      1655
   macro avg       0.83      0.83      0.83      1655
weighted avg       0.83      0.83      0.83      1655


Model: Gradient Boosting
Accuracy: 0.8447
Classification Report:
              precision    recall  f1-score   support

    negative       0.84      